# ML-5333 Project 2 - MLB Data Collection

In [1]:
import pandas as pd
import numpy as np
import baseball_scraper
import pybaseball
import statsapi
import mlbgame

Get masterfile for statsapi team name, code, etc.

In [11]:
with open('data\\master\\team_names.txt', 'r') as file:
    team_list = [x.replace('\n','') for x in file.readlines()]
team_df = []
for team in team_list:
    team_df.append(statsapi.lookup_team(team)[0])
team_df = pd.DataFrame.from_dict(team_df)
team_df.columns = ['statsapi_id','name','team_code','file_code','team_name','loc_name','short_name']
team_df = team_df.sort_values(by=['statsapi_id'])
team_df = team_df.reset_index(drop=True)
team_df.to_csv('data\\master\\statsapi_teams.csv', index=False)

Function to get schedule dataframe for team in specified season

In [6]:
def get_team_game_list(team_code, season):

    date_dict = {
        2016: {'start': '04/03/2016', 'end': '10/02/2016'},
        2017: {'start': '04/02/2017', 'end': '10/01/2017'},
        2018: {'start': '03/28/2018', 'end': '09/30/2018'},
        2019: {'start': '03/27/2019', 'end': '09/29/2019'},
        2020: {'start': '07/23/2020', 'end': '09/27/2020'},
        2021: {'start': '04/01/2021', 'end': '10/03/2021'}
    }

    game_list = statsapi.schedule(start_date=date_dict[season]['start'],end_date=date_dict[season]['end'],team=team_code)

    schedule_df_list = []
    for game in game_list:
        if game['status'] in ['Final', 'Completed Early: Rain']:
            if team_code == game['home_id']:
                team_status = 'Home'
                opp_status = ('away_name', 'away_id')
                if game['home_score'] > game['away_score']:
                    game_result = 'W'
                else:
                    game_result = 'L'
            else:
                team_status = 'Away'
                opp_status = ('home_name', 'home_id')
                if game['away_score'] > game['home_score']:
                    game_result = 'W'
                else:
                    game_result = 'L'
            schedule_df_list.append([game['game_id'],team_status,game[opp_status[0]],game[opp_status[1]],game_result])
    col_list = ['game_id','home/away','opp_name','opp_code','result']
    return pd.DataFrame(schedule_df_list, columns=col_list)

Get schedule dictionary for 2016-2021 seasons and save

In [7]:
schedule_dict = {}
season_list = [2016,2017,2018,2019,2020,2021]
for season in season_list:
    schedule_dict.update({season: {}})
    for team in team_df['statsapi_id']:
        schedule_dict[season].update({team: get_team_game_list(team, season)})
for season in season_list:
    for team in team_df['statsapi_id']:
        schedule_dict[season][team].to_csv(f'data\\statsapi_season_data\\{season}\\{team}.csv')